# Notebook for Work with Optimization and Pool and Executor instances
For possible application in ModelFitting in brian2tools

In [9]:
import numpy as np
import multiprocessing as mp

from scipy.optimize import differential_evolution, rosen
from nevergrad.optimization import optimizerlib


# Scipy

### Run for one argument

In [10]:
def cube(x):
    return x**3

In [11]:
dd = np.array([1,2,3])
cube(dd)

array([ 1,  8, 27])

In [12]:
differential_evolution(cube, [(-5, 5)], workers=1)

     fun: array([-125.])
     jac: array([74.99999981])
 message: 'Optimization terminated successfully.'
    nfev: 139
     nit: 8
 success: True
       x: array([-5.])

In [13]:
differential_evolution(cube, [(-5, 5)], workers=map, updating='deferred')

     fun: array([-125.])
     jac: array([74.99999981])
 message: 'Optimization terminated successfully.'
    nfev: 259
     nit: 16
 success: True
       x: array([-5.])

### Run With Map

In [14]:
from scipy.optimize._differentialevolution import (DifferentialEvolutionSolver)

In [16]:
bounds = [(1,2), (2,3)]

In [17]:
with mp.Pool(2) as p, DifferentialEvolutionSolver(
    rosen, bounds, updating='deferred', workers=p.map) as solver:
    solver.solve()

In [18]:
solver.x

array([1.41369627, 2.        ])

### Test With Chosen Function
We want to get an array and return a value for multiple parameters

In [19]:
bounds = [(1,2), (-10, 2), (0, 5)]

In [20]:
def cube_arr(x):
    return sum(x[:]**3)

In [21]:
cube_arr(np.array([1,2,3]))

36

In [22]:
with mp.Pool(10) as p, DifferentialEvolutionSolver(
    cube_arr, bounds, updating='deferred', workers=p.map) as solver:
    solver.solve()

In [23]:
solver.x

array([  1., -10.,   0.])

### Try with two parameters

In [24]:
# with mp.Pool(10) as p, DifferentialEvolutionSolver(
#     cube_arr_two, bounds, updating='deferred', workers=p.map) as solver:
#     solver.solve()

### Test Args Tuple is Passed

In [25]:
# test that the args tuple is passed to the cost function properly.
bounds = [(-10, 10)]
args = (1., 2., 3.)

In [26]:
def quadratic(x, *args):
    if type(args) != tuple:
        raise ValueError('args should be a tuple')
    return args[0] + args[1] * x + args[2] * x**2.

In [27]:
result = differential_evolution(quadratic,
                                bounds,
                                args=args,
                                polish=True)
result.fun

array([0.66666667])

In [28]:
# test that the args tuple is passed to the cost function properly.
bounds = [(-7, 10)]
args = (1., 2., 3.)

def add_args(x, *args):
    return args[0] + x

In [29]:
result = differential_evolution(add_args,
                                bounds,
                                args=args,
                                polish=True)
result.fun

array([-6.])

### Two inputs with two boundsb

In [30]:
def ackley(x):
    arg1 = -0.2 * np.sqrt(0.5 * (x[0] ** 2 + x[1] ** 2))
    arg2 = 0.5 * (np.cos(2. * np.pi * x[0]) + np.cos(2. * np.pi * x[1]))

    if x[0]+x[1] > 4.1: #this is the constraint, where you would say a+b+c <=1000
        return -20. * np.exp(arg1) - np.exp(arg2) + 20. + np.e
    else:
        return 1000 #some high value

bounds = [(-5, 5), (-5, 5)]
result = differential_evolution(ackley, bounds)
result.x, result.fun

(array([1.98213994, 2.97313942]), 7.961711274689831)

In [31]:
def test(x):
    arg2 = x[0] - x[1]
    return arg2

In [32]:
bounds = [(-5, 5), (-5, 5)]
result = differential_evolution(test, bounds)
result.x, result.fun

(array([-5.,  5.]), -10.0)

### Two Input Two Bounds and map

In [33]:
with mp.Pool(2) as p, DifferentialEvolutionSolver(
    test, bounds, updating='deferred', workers=p.map) as solver:
    solver.solve()

In [34]:
solver.x

array([-5.,  5.])

In [35]:
test(solver.x)

-10.0

# Nevergrad

In [36]:
from concurrent import futures
from nevergrad.optimization import optimizerlib
from nevergrad import instrumentation as inst

In [37]:
def square(x, y=12):
    return sum((x - .5)**2) + abs(y)

In [38]:
instrum = inst.Instrumentation(inst.var.Array(2), y=inst.var.Array(1).asscalar())


In [39]:
def test(x):
    arg2 = x[0] - x[1]
    return arg2

In [40]:
# optimizer = optimizerlib.OnePlusOne(instrumentation=instrum, budget=100, num_workers=5)
# with futures.ProcessPoolExecutor(max_workers=optimizer.num_workers) as executor:
#     recommendation = optimizer.optimize(square, executor=executor, batch_mode=False)

### Chosen example
bounds = [(-5, 5), (0, 10)] # to be reproduced

In [41]:
# define the variables to be a bounded array
arg1 = inst.var.Array(2).bounded(-5, 5)
arg2 = inst.var.Array(2).bounded(0, 10)

instrum = inst.Instrumentation(arg1)
instrum

Instrumentation(Array(shape=(2,), transforms=[ArctanBound(a_max=5, a_min=-5)]))

In [42]:
# pick the optimization method; budget = number of allowed evaluations
optim = optimizerlib.registry['DE'](instrumentation=instrum, budget=5000)

In [43]:
optim.optimize(test)

Candidate(args=(array([-4.88318617,  4.9631408 ]),), kwargs={}, data=[-27.23709927  86.3544705 ])

In [45]:
optim = optimizerlib.registry['DE'](instrumentation=instrum, budget=10000, num_workers=2)

with futures.ProcessPoolExecutor(max_workers=optim.num_workers) as executor:
    recommendation = optim.optimize(test, executor=executor, batch_mode=True)

/home/alteska/anaconda3/envs/brian_opt/lib/python3.6/site-packages/nevergrad/common/tools.py:168: UserWarning: Ignoring since timer was stopped before starting.
  warnings.warn("Ignoring since timer was stopped before starting.")


In [46]:
recommendation

Candidate(args=(array([-5.,  5.]),), kwargs={}, data=[-1.36595896e+15  9.99311081e+14])

## Conclution: they work on similar principles
### Now let's try to plug them with an acutal NeuronGroup example